<a href="https://colab.research.google.com/github/Natesd05/URS-Linguistic-Justice/blob/main/NER_TrueCaseAnnotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stanfordnlp
import stanfordnlp
stanfordnlp.download('en')
stanfordnlp.download('es')
from itertools import islice

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

100%|██████████| 235M/235M [00:41<00:00, 5.62MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Using the default treebank "es_ancora" for language "es".
Would you like to download the models for: es_ancora now? (Y/n)
y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/es_ancora_models.zip


100%|██████████| 226M/226M [00:41<00:00, 5.39MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/es_ancora_models.zip
Extracting models file for: es_ancora
Cleaning up...Done.


In [ ]:
!pip install stanza

import stanza

class Stanza:

    def __init__(self, file_path):
        # Initilize different variables and lists
        self.word_bank = ""
        self.sentences = []
        self.output = ""
        self.person = []
        self.location = []
        self.organization = []
        self.geopolitical = []
        self.nlp_es = stanza.Pipeline(lang='es' and 'en', processors='tokenize,ner')
        count = 0

        # Read through the file, with option to determine line limit to help with proccessing
        with open(file_path, 'r') as file:
            for line in islice(file, 0, 500):
                line = file.readline()
                self.sentences.append(line)

        # Add all sentences to a word bank
        self.word_bank = ' '.join(self.sentences)

    def trueCase(self):
      # Initilize standfordNLP pipeline, which includes tokenizer, multi-word tokens, and parts of speech tagger
      stf_nlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos')
      doc = stf_nlp(self.word_bank)

      # Print word with POS tag
      print(*[f'word: {word.text+" "}\tupos: {word.upos}\txpos:{word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')

      # If POS tag is PROPN or NNS add to data to be processed for NER
      temp_output = ' '.join([w.text.capitalize() if w.upos in ["PROPN", "NNS"] else w.text for sent in doc.sentences for w in sent.words])
      self.output = self.nlp_es(temp_output)

      for ent in self.output.ents:
        print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in self.output.ents], sep='\n')
        if ent.type == 'PERSON':
            self.person.append(ent.text)
        elif ent.type == 'ORG':
            self.organization.append(ent.text)
        elif ent.type == 'GPE':
            self.geopolitical.append(ent.text)
        elif ent.type == 'LOC':
            self.location.append(ent.text)


      print("Persons:")
      print(self.person)
      print("Organization:")
      print(self.organization)
      print("Geopolitical:")
      print(self.geopolitical)
      print("Location:")
      print(self.location)


STANZA = Stanza("/no_spanish.txt")
print("Number of sentences:", len(STANZA.word_bank))
print("Sample sentences:", STANZA.word_bank[:5])  # Print a few sample sentences

STANZA.trueCase()

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Streaming output truncated to the last 5000 lines.
entity: Lodi	type: PERSON
entity: seven	type: CARDINAL
entity: six	type: CARDINAL
entity: San Diego	type: GPE
entity: Carlos	type: PERSON
entity: third	type: ORDINAL
entity: seventh	type: ORDINAL
entity: Carlos	type: PERSON
entity: first	type: ORDINAL
entity: Gleason	type: PERSON
entity: Carlos	type: PERSON
entity: seven years	type: DATE
entity: Rosie	type: PERSON
entity: Rosie	type: PERSON
entity: Rosie	type: PERSON
entity: six	type: CARDINAL
entity: Carlos	type: PERSON
entity: first	type: ORDINAL
entity: last year	type: DATE
entity: twelve year old	type: DATE
entity: eight	type: DATE
entity: fifteen minutes	type: TIME
entity: about five minutes	type: TIME
entity: five minutes	type: TIME
entity: fifteen seconds	type: TIME
entity: first	type: ORDINAL
entity: three	type: CARDINAL
entity: Carlos	type: PERSON
entity: two	type: CARDINAL
entity: nine	type: CARDINAL
entity: one	type: CARDINAL
entity: this year	type: DATE
entity: Kansas city	